# LISP compiler

# Step 0
## The REPL

In [35]:
import re
import operator as op

def READ(program:str) -> str:
    r = read_str(program)
    return r

def EVAL(ast, env):
    if ast == []:
        return ast
    elif type(ast) == list:
        e = eval_ast(ast, env) 
        o = e[0] # el primero de la lista es el operador (ej +)
        p = e[1::] # el resto son los parámetros (ej, los números)
        if o == 'def!': # si el operador es definicion de funcion
            v = EVAL(p[2], env)
            r = env.set(p[1], p[2])
            return r
        return o(*p) # le pasa los parámetros al operador
    else:
        return eval_ast(ast, env)
    return line

def PRINT(line):
    return(pr_str(line))

def rep(program):
    rep = PRINT(EVAL(READ(program), repl_env))
    return rep

def read_str(program):
    t = tokenize(program)
    r = Reader(t)
    l = read_form(r)
    return l

r = r'''[\s,]*(~@|[\[\]{}()'`~^@]|"(?:\\.|[^\\"])*"|;.*|[^\s\[\]{}('"`,;)]*)'''

def tokenize(program:str) -> list:
    tkns = re.split(r, program)
    tkns_list = []
    for t in tkns:
        if t != '':
            tkns_list.append(t) 
    return tkns_list


class Reader:
    def __init__(self, tokens):
        self.tokens = tokens
        self.position = 0
        self.length = len(tokens)
    def nex(self):
        self.position += 1
    def peek(self):
        current_token = self.tokens[self.position]
        return current_token
    
def read_form(tokens:Reader):
    tkn = tokens.peek()
    l = []
    if tkn == '(':
        t = read_list(tokens)
    else: 
        t = read_atom(tkn)
    return t

def read_list(tokens):
    result = []
    tokens.nex() # skips the "(" that brought us here
    tkn = tokens.peek()
    while tkn != ')':
        t = read_form(tokens)
        result.append(t)
        tokens.nex()
        tkn = tokens.peek()
    return result

def read_atom(token):
    try: return int(token)
    except ValueError:
        try: return float(token)
        except ValueError:
            return str(token)
        
def pr_str(token): # vuelve del programa a una string
    l = []
    if type(token) == list:
        for i in token:
            l.append(pr_str(i))
        l = ' '.join(l)
        l = '(' + l + ')'
        return l
    else:
        s = str(token)
        return s
    
def eval_ast(ast, env): # ast = abstract syntax tree = mal data type
    if type(ast) == list:
        l = []
        for i in ast:
            i = EVAL(i, env)
            l.append(i)
        return l
    elif type(ast) == str:
        return env.get(ast)
    else:
        return ast

class Env:
    def __init__(self, outer):
        self.outer = outer
        self.data = {}
    def set(self, symbol, mal_value):
        self.data[symbol] = mal_value
    def find(self, symbol):
        if symbol in self.data:
            return self.data
        elif self.outer is not None:
            return self.outer.find(symbol)
    def get(self, symbol):
        env_with_symbol = self.find(symbol)
        value = env_with_symbol[symbol]
        return value #  If no key is found up the outer chain, then throws/raises a "not found" error.
    
repl_env = Env(None)

repl_env.set('+', op.add)
repl_env.set('-', op.sub)
repl_env.set('*', op.mul)
repl_env.set('/', op.truediv)


In [37]:
repl_env.set('**', op.pow) 
#two asterisks now means power

program = '(def! 5 3)'
rep(program)

TypeError: 'NoneType' object is not subscriptable